In [11]:
# Import Dependancies
from citipy import citipy 
import os
from config import weather_api_key
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import requests
import time

In [12]:
# Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [13]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [14]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [15]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

762

In [16]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

## Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"] 
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")    

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | bredasdorp
Processing Record 2 of Set 1 | evensk
Processing Record 3 of Set 1 | attawapiskat
City not found. Skipping...
Processing Record 4 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 5 of Set 1 | sebezh
Processing Record 6 of Set 1 | narsaq
Processing Record 7 of Set 1 | atuona
Processing Record 8 of Set 1 | myaundzha
Processing Record 9 of Set 1 | vila velha
Processing Record 10 of Set 1 | port elizabeth
Processing Record 11 of Set 1 | nikolskoye
Processing Record 12 of Set 1 | hirara
Processing Record 13 of Set 1 | aykhal
Processing Record 14 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 15 of Set 1 | hermanus
Processing Record 16 of Set 1 | qaanaaq
Processing Record 17 of Set 1 | ushuaia
Processing Record 18 of Set 1 | turtas
Processing Record 19 of Set 1 | tumannyy
City not found. Skipping...
Processing Record 20 of Set 1 | kapaa
Processing

In [17]:
len(city_data)

704

In [18]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Bredasdorp,-34.5322,20.0403,56.46,66,100,9.55,overcast clouds
1,Evensk,61.9500,159.2333,59.25,45,20,3.65,few clouds
2,Sebezh,56.2955,28.4654,64.04,73,4,5.32,clear sky
3,Narsaq,60.9167,-46.0500,62.82,61,56,4.85,broken clouds
4,Atuona,-9.8000,-139.0333,77.25,77,4,14.61,clear sky
5,Myaundzha,63.0494,147.1858,56.44,68,98,1.57,overcast clouds
6,Vila Velha,-20.3297,-40.2925,76.95,80,20,10.36,few clouds
7,Port Elizabeth,-33.9180,25.5701,59.31,75,100,4.00,overcast clouds
8,Nikolskoye,59.7035,30.7861,71.49,71,20,6.55,few clouds
9,Hirara,24.8000,125.2833,84.81,78,77,15.37,broken clouds


In [19]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")